In [ ]:
import glob
import os
import re

import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from PIL import Image

In [ ]:
!pip install seaborn

In [ ]:
base_dir = 'data/jellyfish'

## 1. Was für Daten haben wir eigentlich bekommen?

- Welche Unterordner gibt es?
- Wie viele Dateien gibt es in jedem Unterordner?
- Was hat es mit dem Train_Test_Valid Ordner auf sich?
- Was sind `aug-xx-yy.jpg` für dateien?

### Hinweise
1. `glob` is ein Modul um *pathnames* zu finden.
Dabei übergibt man einen eingachen filter-string (keine regex), wobei '*' ein Platzhalter für alles ist.
Beispiel `glob.glob('data/jellyfish/*')` findet alle Unterordner. Ein filter der auf `*.jpg` endet findet alle jpg Pfadnamen.

2. Einfaches console-print funktioniert ähnlich wie in javascript.
Hier mit der `print(...)` Funktion.
```python
s = f'Eine variable {var_x}, länge einer Liste {len(values)}'
# Quasi gleich:
print('Eine Variable', var_x, 'länge einer Liste', len(values))
print(f'Eine variable {var_x}, länge einer Liste {len(values)}')
```
    Als alternative mit einer regex gibt es die Kombination aus os.listdir(path) und re.search(r'^(0-9)+\.jpg'). Da braucht es dann ggf. noch mehr.

3. Von allen Programmiersprachen kennt ChatGPT Python am besten.

4. alternativ kann man die Fragen auf manuell über den file explorer beantworten

In [ ]:
subfolders = glob.glob(base_dir + '/*')
for s in subfolders:
    files_in_folder = glob.glob(s + '/*')
    print(f'subfolder {s} with {len(files_in_folder)} ')

In [ ]:
train_test_val_subfolder = base_dir + '/Train_Test_Valid'
for stage in ['Train', 'valid', 'test']:
    print('Stage:\t', stage)
    p = train_test_val_subfolder + '/' + stage
    subfolders = glob.glob(p + '/*')

    for s in subfolders:
        files_in_folder = glob.glob(s + '/*')
        print(f'subfolder {s} with {len(files_in_folder)} ')

In [ ]:
p = 'data/jellyfish/Train_Test_Valid/Train/compass_jellyfish'
content = os.listdir(p)
aug_files = [s for s in content if re.match(r'.*aug.*$', s)]

print('Augmented (train) compass jellyfish', len(aug_files))

## Lösung:

- Ein Unterordner pro Klasse. Enthält allerdings nur die Trainingsdaten
- 150 Train daten pro Klasse, teilweise augmented. Anzahl rohdaten leicht unterschiedlich.
- Train_Test_Val Ordner gibt eine Datenunterteilung vor. Könnte man neu mixen.
- 'aug'-Datein sind einfache augmentation- Nur Spiegelung. Keine Rotation, sheering, pixel erasing, color shifting etc.

## 2. Ein kurzer Blick auf die Bilder
- Plotte 3 Bilder pro Klasse
- Wie groß sind die Bilder im Durchschnitt (pro Klasse)?
- Wie ist die Verteilung der Bildgröße (pro Klasse)?
- Bonus: Wie ist das Farbprofil der Bilder

### Hinweise:
- Pillow images kann man mit matplotlib anzeigen
```
# Create a figure with three subplots in a single row
fig, axes = plt.subplots(1, 3, figsize=(15, 5))

# Display each image in a subplot
axes[0].imshow(image1)
axes[0].axis('off')
axes[0].set_title('Image 1')

axes[1].imshow(image2)
axes[1].axis('off')
axes[1].set_title('Image 2')

axes[2].imshow(image3)
axes[2].axis('off')
axes[2].set_title('Image 3')

plt.suptitle('Three Images in a Row', fontsize=16)
```
- Es reicht erstmal nur die train Daten anzuschauen
- Um Bilder einzulesen benutzen wir Pillow.
  Ein Bild öffnet man so: `Image.open(path)`. Ein Image hat dann eine `size, width, height` properties
- ...könnte man auch alles manuell machen.

In [ ]:
train_subfolder = base_dir + '/Train_Test_Valid/Train'
class_folders = glob.glob(train_subfolder + '/*')

def average_color(image):
    # Convert the image to a NumPy array
    img_array = np.array(image)

    # Calculate the average color for each channel (R, G, B)
    avg_color = np.mean(img_array, axis=(0, 1))

    return avg_color

In [ ]:
for folder in class_folders:
    content = glob.glob(folder + '/*')

    
    i0 = Image.open(content[0])
    i1 = Image.open(content[1])
    i2 = Image.open(content[2])
    
    # Create a figure with three subplots in a single row
    fig, axes = plt.subplots(1, 3, figsize=(15, 5))
    
    
    # Display each image in a subplot
    axes[0].imshow(i0)
    axes[0].axis('off')
    axes[0].set_title('Image 1')
    
    axes[1].imshow(i1)
    axes[1].axis('off')
    axes[1].set_title('Image 2')
    
    axes[2].imshow(i2)
    axes[2].axis('off')
    axes[2].set_title('Image 3')
    
    plt.suptitle(folder, fontsize=16)

In [ ]:
# Shape
for folder in class_folders:
    content = glob.glob(folder + '/*')
    widths = []
    heights = []
    sizes = []
    for file in content:
        i = Image.open(file)
        w, h = i.size
        widths.append(w)
        heights.append(h)
        sizes.append(w*h)
    mean_size = np.mean(sizes)
    mean_width = np.mean(widths)
    mean_height = np.mean(heights)
    print(folder, mean_size, mean_width, mean_height)
    print('min/max width', np.min(widths), np.max(widths))
    print('min/max height', np.min(heights), np.max(heights))

    sns.histplot(widths, bins=30, kde=False, color='skyblue')
    plt.title('Histogram using Seaborn')
    plt.xlabel('width')
    plt.ylabel('Frequency')
    plt.show()

In [ ]:
# Color
for folder in class_folders:
    content = glob.glob(folder + '/*')
    total_red = []
    total_green = []
    total_blue = []
    for file in content:
        i = Image.open(file)
        avg_color = average_color(i)

        total_red += [avg_color[0]]
        total_green += [avg_color[1]]
        total_blue += [avg_color[2]]

    plt.figure(figsize=(15, 5))
    plt.subplot(1, 3, 1)
    sns.histplot(total_red, bins=30, kde=False, color='skyblue')
    plt.title('red')

    plt.subplot(1, 3, 2)
    sns.histplot(total_green, bins=30, kde=False, color='skyblue')
    plt.title('green')

    plt.subplot(1, 3, 3)
    sns.histplot(total_blue, bins=30, kde=False, color='skyblue')
    plt.title('blue')

    # Adjust layout and show the plot
    plt.tight_layout()
    plt.show()

## Lösung

- Quallen haben markante Merkmale
- Bilder erscheinen Quadratisch mit einer Größe von etwa 200px
- Bilder sind gleich groß. Mit wenigen (einheitlichen ausreißern)
- Quallen haben unterschiedliche Faben (duh)